<a href="https://colab.research.google.com/github/VesleAnne/Diploma/blob/main/Notebooks/%D0%98%D0%BD%D1%84%D0%B5%D1%80%D0%B5%D0%BD%D1%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Perform Google Colab installs
import os

if "COLAB_GPU" in os.environ:
    print("[INFO] Running in Google Colab, installing requirements.")
    !pip install -U torch -q
    !pip install torch -q
    !pip install tqdm -q # for progress bars
    !pip install sentence-transformers -q # for embedding models
    !pip install accelerate -q # for quantization model loading
    !pip install bitsandbytes -q # for quantizing models (less storage space)
    !pip install python-Levenshtein

[INFO] Running in Google Colab, installing requirements.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 54.7 MB/s eta 0:00:00


In [ ]:
import random
import pandas as pd
import torch
from tqdm.auto import tqdm
import re
import numpy as np
from sentence_transformers import util, SentenceTransformer

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM
from transformers import BitsAndBytesConfig
import textwrap
import random

In [ ]:
import Levenshtein as lev
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
%cd '/content/drive/MyDrive/МФТИ/вкр/RAG'

/content/drive/MyDrive/МФТИ/вкр/RAG


In [ ]:
current_directory = os.getcwd()

### Инференс моделей с DistilBert



In [ ]:
embeddings_df_path = os.path.join(current_directory, 'DATA', 'text_chunks_and_embeddings_df3.csv')
# Convert embedding column back to np.array (it got converted to string when it got saved to CSV)
text_chunks_and_embedding_df = pd.read_csv(embeddings_df_path)

In [ ]:
# Convert embedding column back to np.array (it got converted to string when it got saved to CSV)
text_chunks_and_embedding_df["embedding"] = text_chunks_and_embedding_df["embedding"].apply(lambda x: np.fromstring(x.strip("[]"), sep=" "))

# Convert texts and embedding df to list of dicts
pages_and_chunks = text_chunks_and_embedding_df.to_dict(orient="records")

# Convert embeddings to torch tensor and send to device (note: NumPy arrays are float64, torch tensors are float32 by default)
embeddings = torch.tensor(np.array(text_chunks_and_embedding_df["embedding"].tolist()), dtype=torch.float32).to(device)

In [ ]:
# Define helper function to print wrapped text
def print_wrapped(text, wrap_length=80):
    wrapped_text = textwrap.fill(text, wrap_length)
    print(wrapped_text)

In [ ]:
embedding_model = SentenceTransformer(model_name_or_path="FractalGPT/SbertDistil",
                                      device=device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.58k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/706k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/482k [00:00<?, ?B/s]

In [ ]:
def retrieve_relevant_resources(query: str,
                                embeddings: torch.tensor,
                                model: SentenceTransformer=embedding_model,
                                n_resources_to_return: int=5,
                                print_time: bool=True):
    """
    Embeds a query with model and returns top k scores and indices from embeddings.
    """

    # Embed the query
    query_embedding = model.encode(query,
                                   convert_to_tensor=True)

    # Get cosine similarity scores on embeddings
    cos_sim_scores = util.cos_sim(a=query_embedding, b=embeddings)[0]

    scores, indices = torch.topk(input=cos_sim_scores,
                                 k=n_resources_to_return)

    return scores, indices

def print_top_results_and_scores(query: str,
                                 embeddings: torch.tensor,
                                 pages_and_chunks: list[dict]=pages_and_chunks,
                                 n_resources_to_return: int=5):
    """
    Takes a query, retrieves most relevant resources and prints them out in descending order.

    """

    scores, indices = retrieve_relevant_resources(query=query,
                                                  embeddings=embeddings,
                                                  n_resources_to_return=n_resources_to_return)

    print(f"Query: {query}\n")
    print("Results:")
    # Loop through zipped together scores and indicies
    for score, index in zip(scores, indices):
        print(f"Score: {score:.4f}")
        # Print relevant sentence chunk (since the scores are in descending order, the most relevant chunk will be first)
        print_wrapped(pages_and_chunks[index]["sentence_chunk"])

        print("\n")

NameError: name 'embedding_model' is not defined

## Сайга


In [ ]:
base_model = "IlyaGusev/saiga_llama3_8b"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model, padding_side='right')

tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# bitsandbytes parameters

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

In [ ]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    device_map='auto',
    quantization_config=bnb_config,
    trust_remote_code=True
)

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/277 [00:00<?, ?B/s]

In [ ]:
def prompt_formatter(query: str,
                     context_items: list[dict]) -> str:
    """
    Augments query with text-based context from context_items.
    """
    # Join context items into one dotted paragraph
    context = "- " + "\n- ".join([item["sentence_chunk"] for item in context_items])

    # Create a base prompt with examples to help the model
    # We could also write this in a txt file and import it in if we wanted.
    base_prompt = """system
    Используй следующий контекст для ответов на вопрос.
    Дай себе возможность подумать, извлекая соответствующие отрывки из контекста перед ответом на вопрос.
    Не возвращай контекст, возвращай только ответ.
    Убедись, что твои ответы как можно более понятны и логичны.
    Не повторяйся.
    Используй следующие примеры для выбора идеального стиля ответа.

user
Добрый вечер! Расскажите, пожалуйста о возможности привязки системы гриндата к мессенджерам, например, к телеграму. Если я захочу создать бота анкетатора (поиск соискателей для устройств на работу), чтобы не регистрировать его самостоятельно в системе, могу я как-то подкрепить бота в телеграмм, чтобы после опроса, записать данные куда-то на ваш сервер и затем уже, после принятия решения, зарегистрировать там нового пользователя.
assistant
Здравствуйте. К сожалению, на данный момент чаты в других мессенджерах, кроме нашего, не поддерживаются.

user
Есть ли интеграция с 1С? Если да, то где можно почитать про то, как подключить 1С
assistant
Интеграции с 1С осуществлялись, однако документации по интеграции с 1С нет. Используйте документацию по API 1С и документацию по модулю ETL на платформе https://docs-public.gdcloud.org/platform/ru/%D0%B8%D0%BD%D1%82%D0%B5%D0%B3%D1%80%D0%B0%D1%86%D0%B8%D1%8F-%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85.html

user
Есть ли возможность менять задачи местами в списке драг енд дропом
assistant
К сожалению, менять местами задачи в списке драг енд дропом не получится.

user
Добрый день. Хотел бы узнать, есть ли у вас свободные вакансии без опыта работы?
assistant
Добрый день. Контакты отдела кадров и актуальную информацию о вакансиях можно посмотреть на сайте: https://hh.ru/employer/1993100

user
Позови оператора
assistant
Переключаю на оператора.

user
как сделать так, чтобы при каждом запуске ETL процесса, старые данные удалялись и оставались только актуальные, которые только что были получены?
assistant
В ETL на этапе сохранения нужно настроить тип связи приемника "Альтернативный ключ". В типе объекта, в который сохраняются данные, нужно задать ключ, по которому будут проверяться данные.
\nИспользуй следующий контекст для ответа на вопрос:
{context}
\nРелевантные цитаты: <extract relevant passages from the context here>
User query: {query}

Answer:"""

    # Update base prompt with context items and query
    base_prompt = base_prompt.format(context=context, query=query)

    return base_prompt

In [ ]:
def ask(query,
        temperature=0.3,
        max_new_tokens=512,
        format_answer_text=True,
        return_answer_only=True):
    """
    Takes a query, finds relevant resources/context and generates an answer to the query based on the relevant resources.
    """

    # Get just the scores and indices of top related results
    scores, indices = retrieve_relevant_resources(query=query,
                                                  embeddings=embeddings)

    # Create a list of context items
    context_items = [pages_and_chunks[i] for i in indices]

    # Add score to context item
    for i, item in enumerate(context_items):
        item["score"] = scores[i].cpu() # return score back to CPU

    # Format the prompt with context items
    prompt = prompt_formatter(query=query,
                              context_items=context_items)

    # Tokenize the prompt
    input_ids = tokenizer(prompt, return_tensors="pt").to(device)

    # Generate an output of tokens
    outputs = model.generate(**input_ids,
                                 temperature=temperature,
                                 do_sample=True,
                                 max_new_tokens=max_new_tokens)

    # Turn the output tokens into text
    output_text = tokenizer.decode(outputs[0])

    if format_answer_text:

    # Remove the entire prompt
        if "Answer:" in output_text:
          output_text = output_text.split("Answer:")[1].strip()
        elif "assistant" in output_text:
          output_text = output_text.split("assistant ")[0].strip()
        elif "User query:" in output_text:
          output_text = output_text.split("User query:")[0].strip()
        elif "System response:" in output_text:
          output_text = output_text.split("System response:")[0].strip()

        else:
          # Handle the case when "Answer:" is not found
          # You can choose what to do in this case, for example, set output_text to an empty string
          output_text = ""

        # Replace special tokens and unnecessary help message
        output_text = output_text.replace(prompt, "").replace("<bos>", "").replace("<eos>", "").replace("<|begin_of_text|> ", "").replace("<|eot_id|>", "").replace("", "").replace("", "").replace("Sure, here is the answer to the user query:\n\n", "").replace("end  assistant", "")

        # Remove the prompt at the beginning
        output_text = output_text.replace(prompt_formatter(query, context_items), "")

        output_text = output_text.replace(prompt, "").replace(prompt_formatter(query, context_items), "")

    # Only return the answer without the context items
    if return_answer_only:
        return output_text

    return output_text, context_items

In [ ]:
def add_model_column(dataset, model, tokenizer, model_column_name):
    """
    Проходит по датасету и добавляет ответы модели в новый столбец.

    Args:
    - dataset (pd.DataFrame): Датасет с вопросами.
    - model: Модель для генерации ответов.
    - tokenizer: Токенизатор модели.
    - model_column_name (str): Имя нового столбца, куда будут добавлены ответы модели.

    Returns:
    - pd.DataFrame: Датасет с добавленным столбцом ответов модели.
    """
    # Создаем пустой столбец для ответов модели
    dataset[model_column_name] = ""

    # Проходим по каждой строке в датасете
    for index, row in dataset.iterrows():
        # Получаем вопрос из текущей строки
        query = row["question"]


        answer, context_items = ask(query=query,
                            temperature=0.6,
                            max_new_tokens=256,
                            return_answer_only=False)

        # Добавляем ответ модели в соответствующую строку столбца
        dataset.at[index, model_column_name] = answer

    return dataset

In [ ]:
def compute_similarity(dataset, model_column_name, answer_column_name="answer A"):
    """
    Сравнивает ответы из столбцов answer A и model и вычисляет процент сходства.

    Args:
    - dataset (pd.DataFrame): Датасет с ответами модели и эталонными ответами.
    - saiga_column_name (str): Имя столбца с ответами модели.
    - answer_column_name (str): Имя столбца с эталонными ответами.

    Returns:
    - pd.DataFrame: Датасет с новым столбцом, содержащим процент сходства между ответами.
    """
    # Создаем новый столбец для процента сходства
    dataset["similarity_percentage"] = ""

    # Проходим по каждой строке в датасете
    for index, row in dataset.iterrows():
        # Получаем ответы из столбцов
        model_answer = row[model_column_name]
        true_answer = row[answer_column_name]

        # Вычисляем расстояние Левенштейна между ответами
        distance = lev.distance(model_answer, true_answer)

        # Вычисляем процент сходства на основе расстояния Левенштейна
        similarity_percentage = ((max(len(model_answer), len(true_answer)) - distance) / max(len(model_answer), len(true_answer))) * 100

        # Добавляем процент сходства в соответствующую строку нового столбца
        dataset.at[index, "similarity_percentage"] = similarity_percentage

    return dataset

In [ ]:
def compute_bleu_scores(dataset, reference_column, candidate_column):
    """
    Вычисляет метрику BLEU между эталонным текстом и сгенерированным текстом для каждой строки датасета.

    Args:
    - dataset (pd.DataFrame): Датасет с эталонными и сгенерированными ответами.
    - reference_column (str): Имя столбца с эталонными ответами.
    - candidate_column (str): Имя столбца с сгенерированными ответами.

    Returns:
    - pd.DataFrame: Датасет с добавленным столбцом BLEU-метрик для каждой строки.
    """

    def compute_bleu(reference, candidate):
        smoothing = SmoothingFunction().method7  # Выберите метод сглаживания, который наилучшим образом подходит для вашей задачи
        return sentence_bleu([reference.split()], candidate.split(), smoothing_function=smoothing)

    # Создаем пустой список для хранения метрик BLEU
    bleu_scores = []

    # Проходим по каждой строке в датасете
    for index, row in dataset.iterrows():
        # Получаем эталонный и сгенерированный ответы из соответствующих столбцов
        reference_answer = row[reference_column]
        candidate_answer = row[candidate_column]

        # Вычисляем BLEU метрику для текущей строки
        bleu_score = compute_bleu(reference_answer, candidate_answer)

        # Добавляем BLEU метрику в список
        bleu_scores.append(bleu_score)

    # Добавляем столбец BLEU-метрик в датасет
    dataset['BLEU_Score'] = bleu_scores

    return dataset

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/МФТИ/вкр/RAG/DATA/testdata.xlsx')

In [ ]:
df.head()

,question,answer A
0,Как исправить ошибку в динамической структуре ?,"Скорее всего, данные сохраняются в другой эле..."
1,"Как сделать так, чтобы при каждом запуске ETL ...",В ETL на этапе сохранения нужно настроить тип ...
2,"Подскажите, пожалуйста, а как со стартовой стр...",Нужно снова открыть сайт Ссылка на приложение_...
3,"Добрый день! У меня алгоритм выдает ""Ошибка ва...","Здравствуйте, такая ошибка бывает, если неправ..."
4,"Я создала пользователей в приложении, дала им ...","Попробуйте, пожалуйста, зайти в нового пользов..."


In [ ]:
# Добавляем столбец с ответами модели
model_column_name = "Sbert+Saiga"
df_with_model = add_model_column(df, model=model, tokenizer=tokenizer, model_column_name=model_column_name)

In [ ]:
df_with_model.head()

,question,answer A,Sbert+Saiga
0,Как исправить ошибку в динамической структуре ?,"Скорее всего, данные сохраняются в другой эле...","Возможно, вы хотите сказать, что вам нужна пом..."
1,"Как сделать так, чтобы при каждом запуске ETL ...",В ETL на этапе сохранения нужно настроить тип ...,В ETL на этапе сохранения нужно настроить тип ...
2,"Подскажите, пожалуйста, а как со стартовой стр...",Нужно снова открыть сайт Ссылка на приложение_...,Для возврата на стартовую страницу (список про...
3,"Добрый день! У меня алгоритм выдает ""Ошибка ва...","Здравствуйте, такая ошибка бывает, если неправ...",Для начала необходимо определить какие конкрет...
4,"Я создала пользователей в приложении, дала им ...","Попробуйте, пожалуйста, зайти в нового пользов...","1) Проверьте, правильно ли заполнена форма вхо..."


In [ ]:
# Вычисляем процент сходства
df_with_similarity = compute_similarity(df_with_model, model_column_name=model_column_name)
df_with_similarity

,question,answer A,Sbert+Saiga,similarity_percentage
0,Как исправить ошибку в динамической структуре ?,"Скорее всего, данные сохраняются в другой эле...",Для исправления ошибок в динамической структур...,9.792844
1,"Как сделать так, чтобы при каждом запуске ETL ...",В ETL на этапе сохранения нужно настроить тип ...,В ETL на этапе сохранения нужно настроить тип ...,37.106918
2,"Подскажите, пожалуйста, а как со стартовой стр...",Нужно снова открыть сайт Ссылка на приложение_...,"Чтобы просмотреть список проектов, в которых п...",17.804154
3,"Добрый день! У меня алгоритм выдает ""Ошибка ва...","Здравствуйте, такая ошибка бывает, если неправ...",Выделите основные элементы запроса: пользовате...,9.45122
4,"Я создала пользователей в приложении, дала им ...","Попробуйте, пожалуйста, зайти в нового пользов...","Сначала нужно убедиться, что пользователь дейс...",6.13563
5,"Добрый вечер! Расскажите, пожалуйста о возможн...","К сожалению, на данный момент чаты в других ме...","Здравствуйте. К сожалению, на данный момент ча...",45.0
6,"Скажите еще, пожалуйста. Есть ли интеграция с ...","Интеграции с 1С осуществлялись, однако докумен...","Да, интеграции с 1С осуществлялись, однако док...",40.379404
7,"А как сделать так, чтобы при каждом запуске ET...",В ETL на этапе сохранения нужно настроить тип ...,В ETL на этапе сохранения нужно настроить тип ...,26.818182
8,"Подскажите, пожалуйста, как отобразить объект ...",Чтобы отобразить список задач для сотрудника м...,"Вот базовый пример кода, который вы можете исп...",9.090909
9,"И еще скажите, пожалуйста, есть ли возможность...",Менять местами задачи в списке драг енд дропом...,"К сожалению, менять местами задачи в списке др...",44.360902


In [ ]:
# Вычисляем BLEU-метрики
df_with_bleu_scores = compute_bleu_scores(dataset=df_with_similarity, reference_column="answer A", candidate_column=model_column_name)
df_with_bleu_scores.head(10)

,question,answer A,Sbert+Saiga,similarity_percentage,BLEU_Score
0,Как исправить ошибку в динамической структуре ?,"Скорее всего, данные сохраняются в другой эле...",Для исправления ошибок в динамической структур...,9.792844,0.069392
1,"Как сделать так, чтобы при каждом запуске ETL ...",В ETL на этапе сохранения нужно настроить тип ...,В ETL на этапе сохранения нужно настроить тип ...,37.106918,0.466606
2,"Подскажите, пожалуйста, а как со стартовой стр...",Нужно снова открыть сайт Ссылка на приложение_...,"Чтобы просмотреть список проектов, в которых п...",17.804154,0.074307
3,"Добрый день! У меня алгоритм выдает ""Ошибка ва...","Здравствуйте, такая ошибка бывает, если неправ...",Выделите основные элементы запроса: пользовате...,9.45122,0.000000
4,"Я создала пользователей в приложении, дала им ...","Попробуйте, пожалуйста, зайти в нового пользов...","Сначала нужно убедиться, что пользователь дейс...",6.13563,0.069320
5,"Добрый вечер! Расскажите, пожалуйста о возможн...","К сожалению, на данный момент чаты в других ме...","Здравствуйте. К сожалению, на данный момент ча...",45.0,0.477735
6,"Скажите еще, пожалуйста. Есть ли интеграция с ...","Интеграции с 1С осуществлялись, однако докумен...","Да, интеграции с 1С осуществлялись, однако док...",40.379404,0.379520
7,"А как сделать так, чтобы при каждом запуске ET...",В ETL на этапе сохранения нужно настроить тип ...,В ETL на этапе сохранения нужно настроить тип ...,26.818182,0.343095
8,"Подскажите, пожалуйста, как отобразить объект ...",Чтобы отобразить список задач для сотрудника м...,"Вот базовый пример кода, который вы можете исп...",9.090909,0.077093
9,"И еще скажите, пожалуйста, есть ли возможность...",Менять местами задачи в списке драг енд дропом...,"К сожалению, менять местами задачи в списке др...",44.360902,0.500453


In [ ]:
df_with_bleu_scores.to_excel('Sbert+Saiga.xlsx', index=False)

## Альпака

In [ ]:
base_model = "IlyaGusev/fred_t5_ru_turbo_alpaca"
tokenizer = AutoTokenizer.from_pretrained(base_model, padding_side='right')

tokenizer_config.json:   0%|          | 0.00/914 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.76M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.74k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    device_map='auto',
    quantization_config=bnb_config,
    trust_remote_code=True
)

config.json:   0%|          | 0.00/869 [00:00<?, ?B/s]

ValueError: Unrecognized configuration class <class 'transformers.models.t5.configuration_t5.T5Config'> for this kind of AutoModel: AutoModelForCausalLM.
Model type should be one of BartConfig, BertConfig, BertGenerationConfig, BigBirdConfig, BigBirdPegasusConfig, BioGptConfig, BlenderbotConfig, BlenderbotSmallConfig, BloomConfig, CamembertConfig, LlamaConfig, CodeGenConfig, CohereConfig, CpmAntConfig, CTRLConfig, Data2VecTextConfig, DbrxConfig, ElectraConfig, ErnieConfig, FalconConfig, FuyuConfig, GemmaConfig, GitConfig, GPT2Config, GPT2Config, GPTBigCodeConfig, GPTNeoConfig, GPTNeoXConfig, GPTNeoXJapaneseConfig, GPTJConfig, JambaConfig, JetMoeConfig, LlamaConfig, MambaConfig, MarianConfig, MBartConfig, MegaConfig, MegatronBertConfig, MistralConfig, MixtralConfig, MptConfig, MusicgenConfig, MusicgenMelodyConfig, MvpConfig, OlmoConfig, OpenLlamaConfig, OpenAIGPTConfig, OPTConfig, PegasusConfig, PersimmonConfig, PhiConfig, Phi3Config, PLBartConfig, ProphetNetConfig, QDQBertConfig, Qwen2

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/МФТИ/вкр/RAG/DATA/testdata.xlsx')
# Добавляем столбец с ответами модели
model_column_name = "Sbert+Alpaca"
df_with_model = add_model_column(df, model=model, tokenizer=tokenizer, model_column_name=model_column_name)

In [ ]:
# Вычисляем процент сходства
df_with_similarity = compute_similarity(df_with_model, model_column_name=model_column_name)
df_with_similarity
# Вычисляем BLEU-метрики
df_with_bleu_scores = compute_bleu_scores(dataset=df_with_similarity, reference_column="answer A", candidate_column=model_column_name)
df_with_bleu_scores.head()
df_with_bleu_scores.to_csv('Sbert+Alpaca.csv', index=False)

## Вихрь

In [ ]:
base_model = "Vikhrmodels/Vikhr-7B-instruct_0.4"
tokenizer = AutoTokenizer.from_pretrained(base_model, padding_side='right')

tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    device_map='auto',
    quantization_config=bnb_config,
    trust_remote_code=True
)

tokenizer_config.json:   0%|          | 0.00/3.86k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.64M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/5.75M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/622 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.71G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/648M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/133 [00:00<?, ?B/s]

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/МФТИ/вкр/RAG/DATA/testdata.xlsx')
# Добавляем столбец с ответами модели
model_column_name = "Sbert+Vikhr"
df_with_model = add_model_column(df, model=model, tokenizer=tokenizer, model_column_name=model_column_name)
# Вычисляем процент сходства
df_with_similarity = compute_similarity(df_with_model, model_column_name=model_column_name)
df_with_similarity
# Вычисляем BLEU-метрики
df_with_bleu_scores = compute_bleu_scores(dataset=df_with_similarity, reference_column="answer A", candidate_column=model_column_name)
df_with_bleu_scores.head()
df_with_bleu_scores.to_csv('Sbert+Vikhr.csv', index=False)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

## RuGpt

In [ ]:
base_model = "ai-forever/ruGPT-3.5-13B"
tokenizer = AutoTokenizer.from_pretrained(base_model, padding_side='right')

tokenizer_config.json:   0%|          | 0.00/933 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.77M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/582 [00:00<?, ?B/s]

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    device_map='auto',
    quantization_config=bnb_config,
    trust_remote_code=True
)

config.json:   0%|          | 0.00/782 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/42.1k [00:00<?, ?B/s]

pytorch_model-00001-of-00006.bin:   0%|          | 0.00/9.92G [00:00<?, ?B/s]

pytorch_model-00002-of-00006.bin:   0%|          | 0.00/9.68G [00:00<?, ?B/s]

pytorch_model-00003-of-00006.bin:   0%|          | 0.00/9.68G [00:00<?, ?B/s]

pytorch_model-00004-of-00006.bin:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

pytorch_model-00005-of-00006.bin:   0%|          | 0.00/9.79G [00:00<?, ?B/s]

pytorch_model-00006-of-00006.bin:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/МФТИ/вкр/RAG/DATA/testdata.xlsx')
# Добавляем столбец с ответами модели
model_column_name = "Sbert+ruGPT"

In [ ]:
df_with_model = add_model_column(df, model=model, tokenizer=tokenizer, model_column_name=model_column_name)
# Вычисляем процент сходства
df_with_similarity = compute_similarity(df_with_model, model_column_name=model_column_name)
df_with_similarity
# Вычисляем BLEU-метрики
df_with_bleu_scores = compute_bleu_scores(dataset=df_with_similarity, reference_column="answer A", candidate_column=model_column_name)
df_with_bleu_scores.head(7)

,question,answer A,Sbert+ruGPT,similarity_percentage,BLEU_Score
0,Как исправить ошибку в динамической структуре ?,"Скорее всего, данные сохраняются в другой эле...",User query: Как исправить ошибку в динамическо...,25.806452,0.037757
1,"Как сделать так, чтобы при каждом запуске ETL ...",В ETL на этапе сохранения нужно настроить тип ...,User query:,0.510204,0.000000
2,"Подскажите, пожалуйста, а как со стартовой стр...",Нужно снова открыть сайт Ссылка на приложение_...,"Добрый день. К сожалению, на данный момент чат...",9.176225,0.068291
3,"Добрый день! У меня алгоритм выдает ""Ошибка ва...","Здравствуйте, такая ошибка бывает, если неправ...","user\nЗдравствуйте! Подскажите, пожалуйста, ес...",10.810811,0.070157
4,"Я создала пользователей в приложении, дала им ...","Попробуйте, пожалуйста, зайти в нового пользов...",На данный момент учетные записи созданы и наст...,6.015779,0.067215
5,"Добрый вечер! Расскажите, пожалуйста о возможн...","К сожалению, на данный момент чаты в других ме...","Здравствуйте. К сожалению, на данный момент ча...",9.635974,0.165741
6,"Скажите еще, пожалуйста. Есть ли интеграция с ...","Интеграции с 1С осуществлялись, однако докумен...","Здравствуйте. К сожалению, на данный момент ча...",10.154125,0.067894


In [ ]:
df_with_bleu_scores.to_csv('Sbert+ ruGPT.csv', index=False)